In [ ]:
import pydoc
import connection_ms_sql 
### Create the Database in Micrsoft SQL SERVER and Then Create the Tables
# Connection parameters
server = 'NASIR'  # e.g., 'localhost\SQLEXPRESS' or an IP address
database = 'Maintenance'  # Connect to the database to create another database or just remove the database from the connection string
username = 'nasir'  # SQL Server username, omit if using Windows authentication
password = 'Cheema01'  # SQL Server password, omit if using Windows authentication
driver  = 'ODBC Driver 17 for SQL Server'
port = '1433'

# Connection string
# For Windows Authentication, use: "Trusted_Connection=yes;" and remove UID and PWD
conn_str = f'DRIVER={driver};SERVER={server};UID={username};PWD={password};Port={port}'
#pyodbc.connect(conn_str)

# Create a new database named 'MyNewDatabase'
database_name = 'Assignment'

# Connect to SQL Server and create New Database
try:
    with pyodbc.connect(conn_str, autocommit=True) as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"CREATE DATABASE {database_name}")
            print(f"Database '{database_name}' created successfully.")
except:
    print(f'Database "{database_name}" already present')
    
# Connection string updated with database name after creation of database
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database_name};UID={username};PWD={password};Port={port}'

# Data types for the training and ideal tables' columns
default_data_type = "FLOAT"

# Special data types for certain columns in the test table
special_data_types = {
    "Delta Y (test func)": "FLOAT",
    "No. of ideal func": "VARCHAR(255)"
}

# Number of Columns in ideal.csv as num_columns is equal to 51 for ideal_column_names generation and the first Column name is 'X'
num_columns = 51
ideal_column_names = ['X'] + [f'Y{i} (ideal func)' for i in range(1, num_columns)]

# Column names for each table
tables = {
    "train_table": ['X', 'Y1 (training func)', 'Y2 (training func)', 'Y3 (training func)', 'Y4 (training func)'],
    "test_table": ['X (test func)', 'Y (test func)', 'Delta Y (test func)', 'No. of ideal func'],
    "ideal_table": ideal_column_names,
}

try:
    with pyodbc.connect(conn_str, autocommit=True) as conn:
        with conn.cursor() as cursor:
            for table_name, columns in tables.items():
                create_table_command = f"CREATE TABLE {table_name} ("
                
                for col in columns:
                    # Determine the data type for the column
                    if table_name == "test_table" and col in special_data_types:
                        # Use special data type for certain test table columns
                        col_data_type = special_data_types[col]
                    else:
                        # Default data type for other columns
                        col_data_type = default_data_type
                    
                    create_table_command += f"[{col}] {col_data_type}, "
                
                create_table_command = create_table_command.rstrip(", ")  # Remove trailing comma
                create_table_command += ")"
            
                # Execute the CREATE TABLE command
                cursor.execute(create_table_command)
                print(create_table_command)
                print(f"Table '{table_name}' created successfully.")
except Exception as e:
    print("An error occurred:", e)

In [ ]:
### Read the data from CSV and then change the column names to match MS SQL Server Tables Created in the previous step and then insert the data into SQL Tables
from sqlalchemy import create_engine
import pandas as pd

server = 'NASIR'  # e.g., 'localhost\SQLEXPRESS' or an IP address
database = 'Assignment'  # Connect to the database to create another database or just remove the database from the connection string
username = 'nasir'  # SQL Server username, omit if using Windows authentication
password = 'Cheema01'  # SQL Server password, omit if using Windows authentication
driver  = 'ODBC Driver 17 for SQL Server'
port = '1433'

# Read Csv files for train test and ideal data with in the same forlder
dataset_path = r'C:\Users\nasir\OneDrive\Desktop\IU-Course Data\PROGRAMMING WITH PYTHON-course2-sem1\3.WRITTEN ASSIGNMENT\written Assignment\Assignment'
dataset_unicode_path = dataset_path.replace('\\','/')
#print(dataset_unicode_path)
train_dataset_filename = 'train.csv'
test_dataset_filename = 'test.csv'
ideal_dataset_filename = 'ideal.csv'

#Create the DataFrame form the csv file using Pandas
df_train = pd.read_csv(f'{dataset_unicode_path}/{train_dataset_filename}')
df_ideal = pd.read_csv(f'{dataset_path}/{ideal_dataset_filename}')
df_test = pd.read_csv(f'{dataset_path}/{test_dataset_filename}')

num_columns = len(df_ideal.columns)
#print(num_columns)
# Defining a new list of column names
train_column_names = ['X', 'Y1 (training func)', 'Y2 (training func)', 'Y3 (training func)', 'Y4 (training func)']
test_column_names = ['X (test func)', 'Y (test func)', 'Delta Y (test func)', 'No. of ideal func']

# Create a list of Ideal Function Coloum Names
ideal_column_names = []
for i in range(num_columns):
    if i == 0:
        ideal_column_names.append('X')
    else:
        ideal_column_names.append(f'Y{i} (ideal func)')
#print(ideal_column_names)

# Assigning the new column names to the DataFrame
df_train.columns = train_column_names
df_ideal.columns = ideal_column_names
df_test.columns = test_column_names[:2]

# Adding two new empty columns in test data frame
df_test[test_column_names[2]] = ''
df_test[test_column_names[3]] = ''

print(df_train)
print(df_ideal)
print(df_test)

### Insert the data into SQL Server
# Your connection string to insert data line by line using sqlalchemy
connection_string_alchemy = f'mssql+pyodbc://{username}:{password}@{server}/{database_name}?driver={driver.replace(" ", "+")}&TrustServerCertificate=yes'
engine = create_engine(connection_string_alchemy)
#print(connection_string_alchemy)

# Insert df_train into train_table
df_train.to_sql('train_table', con=engine, if_exists='replace', index=False)

# Insert df_ideal into ideal_table
df_ideal.to_sql('ideal_table', con=engine, if_exists='replace', index=False)

# Insert df_test into test_table
df_test.to_sql('test_table', con=engine, if_exists='replace', index=False)


In [ ]:
###Read from SQL SERVER and Create Pandas Data Frame form the tables
import pandas as pd
from sqlalchemy import create_engine

# Create MS SQL 'engine' is already created from the previous step
connection_string_alchemy = f'mssql+pyodbc://{username}:{password}@{server}/{database_name}?driver={driver.replace(" ", "+")}&TrustServerCertificate=yes'
engine = create_engine(connection_string_alchemy)
# Read 'train_table' into a DataFrame
query_train = "SELECT * FROM train_table"
df_train = pd.read_sql_query(query_train, engine)

# Read 'ideal_table' into a DataFrame
query_ideal = "SELECT * FROM ideal_table"
df_ideal = pd.read_sql_query(query_ideal, engine)

# Read 'test_table' into a DataFrame
query_test = "SELECT * FROM test_table"
df_test = pd.read_sql_query(query_test, engine)


In [ ]:
import pandas as pd
import numpy as np

# Assuming df_train and df_ideal are defined and loaded with data
# Sort df_train and df_ideal by 'X' if not already sorted
df_train = df_train.sort_values(by='X')
df_ideal = df_ideal.sort_values(by='X')

# Initialize a dictionary to hold the SSD sums for each training function comparison with ideal functions
ssd_sums = {}  # Outer dictionary: keys are training functions, values are ssd_sums1 dictionaries
top_four_ideal_functions = []  # Store the top ideal function for each training function

for j in range(1, 5):  # Loop through each training function
    ssd_sums1 = {}  # Reset ssd_sums for each training function
    for i in range(1, 51):  # Loop through each ideal function
        col_name = f'Y{i} (ideal func)'
        # Calculate SSD for this training function against each ideal function
        ssd = ((df_train.iloc[:, j] - df_ideal[col_name])**2).sum()
        ssd_sums1[col_name] = ssd
    ssd_sums[f'Y{j} (training func)'] = ssd_sums1  # Store ssd_sums for this training function
    # Sort ssd_sums to find the ideal function with the lowest SSD for this training function and append its name directly
    top_four_ideal_functions.append(sorted(ssd_sums1, key=ssd_sums1.get)[0])


print("SSD sums for all comparisons:", ssd_sums)
print("Top ideal function for each training function:", top_four_ideal_functions)
#print("Top four ideal functions for the last training function comparison:", top_four_ideal_functions1)


In [ ]:
# Assuming ssd_sums is structured as shown above

# Extracting individual dictionaries for each training function
ssd_sums_y1 = ssd_sums['Y1 (training func)']
ssd_sums_y2 = ssd_sums['Y2 (training func)']
ssd_sums_y3 = ssd_sums['Y3 (training func)']
ssd_sums_y4 = ssd_sums['Y4 (training func)']

# Example usage: Print the ideal function with the lowest SSD for Y1 (training func)
lowest_ssd_y1 = min(ssd_sums_y1, key=ssd_sums_y1.get)
print(f"The ideal function with the lowest SSD for Y1 (training func) is {lowest_ssd_y1} with an SSD of {ssd_sums_y1[lowest_ssd_y1]}")

# If you need to perform operations or visualizations for each training function's SSD values,
# you can iterate over the ssd_sums dictionary and process each inner dictionary as needed.
for training_func, ssd_values in ssd_sums.items():
    # You can now work with each ssd_values dictionary which corresponds to each training function
    # For example, finding the lowest SSD value for each training function
    lowest_ssd = min(ssd_values, key=ssd_values.get)
    print(f"The ideal function with the lowest SSD for {training_func} is {lowest_ssd} with an SSD of {ssd_values[lowest_ssd]}")
print(ssd_sums)

In [ ]:
print(ssd_sums_y1)

In [ ]:
##Create plot for ssd_sums
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
import numpy as np
def plot(ssd_sums):
    # Function to create a plot for given SSD sums
    def create_ssd_plot(ssd_sums, title):
        # Calculate log10 of SSD values
        ssd_log = np.log10(list(ssd_sums.values()))
        
        # Find the minimum SSD value and its corresponding ideal function
        min_ssd_value = min(ssd_log)
        colors = ['green' if ssd == min_ssd_value else 'lightblue' for ssd in ssd_log]
        
        # Create a ColumnDataSource, now including a color field
        ssd_source_log = ColumnDataSource(data=dict(
            ideal_funcs=list(ssd_sums.keys()),
            ssd=ssd_log,
            colors=colors  # Add color information here
        ))
        
        # Create the figure
        p = figure(x_range=list(ssd_sums.keys()), title=title, width=700, height=350, tools="")
        
        # Draw the bars, setting fill_color to use the colors from the source
        p.vbar(x='ideal_funcs', top='ssd', width=0.9, source=ssd_source_log, line_color='white', fill_color='colors')
        
        # Configure hover tool
        hover = HoverTool()
        hover.tooltips = [
            ("Ideal Function", "@ideal_funcs"),
            ("Log10(SSD)", "@ssd{0,0.00}")
        ]
        p.add_tools(hover)
        
        # Rotate x-axis labels for better visibility
        p.xaxis.major_label_orientation = "vertical"
        
        return p


    # Create a plot for each set of SSD sums
    p1 = create_ssd_plot(ssd_sums['Y1 (training func)'], f'SSD for Y1 (training func)-Log Sclae & The Function selected is {min(ssd_sums["Y1 (training func)"], key=ssd_sums["Y1 (training func)"].get)} @Value {ssd_sums["Y1 (training func)"][min(ssd_sums["Y1 (training func)"], key=ssd_sums["Y1 (training func)"].get)]}')
    p2 = create_ssd_plot(ssd_sums['Y2 (training func)'], f'SSD for Y2 (training func)-Log Sclae & The Function selected is {min(ssd_sums["Y2 (training func)"], key=ssd_sums["Y2 (training func)"].get)} @Value {ssd_sums["Y2 (training func)"][min(ssd_sums["Y2 (training func)"], key=ssd_sums["Y2 (training func)"].get)]}')
    p3 = create_ssd_plot(ssd_sums['Y3 (training func)'], f'SSD for Y3 (training func)-Log Sclae & The Function selected is {min(ssd_sums["Y3 (training func)"], key=ssd_sums["Y3 (training func)"].get)} @Value {ssd_sums["Y3 (training func)"][min(ssd_sums["Y3 (training func)"], key=ssd_sums["Y3 (training func)"].get)]}')
    p4 = create_ssd_plot(ssd_sums['Y4 (training func)'], f'SSD for Y4 (training func)-Log Sclae & The Function selected is {min(ssd_sums["Y4 (training func)"], key=ssd_sums["Y4 (training func)"].get)} @Value {ssd_sums["Y4 (training func)"][min(ssd_sums["Y4 (training func)"], key=ssd_sums["Y4 (training func)"].get)]}')

    # Arrange the plots in a grid
    grid = gridplot([[p1, p2], [p3, p4]])

    # Show the grid
    show(grid)
    
plot(ssd_sums)


In [ ]:
#Calculate deviations
import numpy as np

# Assuming chosen_ideal_functions is a list of column names for the chosen ideal functions from df_ideal
# Example: ['Y1 (ideal func)', 'Y2 (ideal func)', ...]

# Assuming the training function columns are named 'Y1 (training func)', 'Y2 (training func)', etc.
training_function_columns = ['Y1 (training func)', 'Y2 (training func)', 'Y3 (training func)', 'Y4 (training func)']

max_deviations = {}
for ideal_func in top_four_ideal_functions:
    all_deviations = []
    for train_func in training_function_columns:
        # Here, we calculate deviations for each training function against the current ideal function
        deviations = np.abs(df_train[train_func] - df_ideal[ideal_func])
        all_deviations.append(deviations)
    
    # Combine deviations from all training functions for the current ideal function
    combined_deviations = np.concatenate(all_deviations)
    max_deviations[ideal_func] = np.max(combined_deviations)

# Adjust max deviations by factor sqrt(2)
adjustment_factor = np.sqrt(2)
adjusted_deviations = {func: deviation * adjustment_factor for func, deviation in max_deviations.items()}


In [ ]:
#used in test results calculations
def find_best_match(x_val, y_val, chosen_functions, df_ideal, adjusted_deviations):
    best_match = {'func': None, 'deviation': np.inf}
    
    for func in chosen_functions:
        ideal_y_val = df_ideal.loc[df_ideal['X'] == x_val, func].iloc[0]
        deviation = np.abs(ideal_y_val - y_val)
        
        if deviation < adjusted_deviations[func] and deviation < best_match['deviation']:
            best_match = {'func': func, 'deviation': deviation}
    
    return best_match


In [ ]:
# Initialize the list to store results
test_results = []

for index, row in df_test.iterrows():
    match = find_best_match(row['X (test func)'], row['Y (test func)'], top_four_ideal_functions, df_ideal, adjusted_deviations)
    test_results.append({
        'X (test func)': row['X (test func)'],
        'Y (test func)': row['Y (test func)'],
        'Delta Y (test func)': match['deviation'] if match['func'] else None,
        'No. of ideal func': match['func']
        
    })

# Create a new DataFrame from the results
df_test_results = pd.DataFrame(test_results)
df_test_results = df_test_results.sort_values(by='X (test func)')

# If you need to only include rows where an assignment was made, you can filter the DataFrame
df_assigned = df_test_results.dropna(subset=['No. of ideal func'])


In [ ]:
##scater plot for test results
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import numpy as np

# Assuming df_test_results is already defined and loaded with data

# Ensure the notebook output is set correctly
output_notebook()

# Generate a random color for each data point
np.random.seed(42)  # For reproducibility
colors = ['#' + ''.join([np.random.choice(list('0123456789ABCDEF')) for j in range(6)]) for i in range(len(df_test_results))]

# Prepare the data
source = ColumnDataSource(data={
    'x_test': df_test_results['X (test func)'].astype(float),
    'y_test': df_test_results['Y (test func)'].astype(float),
    'ideal_func': df_test_results['No. of ideal func'],
    'delta_y': df_test_results['Delta Y (test func)'],
    'colors': colors  # Add the generated colors to the source
})

# Create the figure
p = figure(width=1000, height=600, title="Test Results Scatter Plot",
           x_axis_label='X (test func)', y_axis_label='Y (test func)',
           tools="pan,wheel_zoom,box_zoom,reset,save")

# Add a scatter renderer with dynamic coloring
p.scatter('x_test', 'y_test', color='colors', source=source, size=20, alpha=0.6)

# Add a hover tool
hover = HoverTool()
hover.tooltips = [
    ("X (test func)", "@x_test"),
    ("Y (test func)", "@y_test"),
    ("No. of ideal func", "@ideal_func"),
    ("Delta Y", "@delta_y")
]

p.add_tools(hover)

# Show the result
show(p)


In [ ]:
print(source.data)


In [ ]:
df_test_results.to_excel('test_results.xlsx', index=False)
df_assigned.to_excel('assigned.xlsx', index=False)

In [ ]:
import pydoc
import pandas as pd
from connection_ms_sql import CreateDatabaseTable as cdt
from read_csv_save_data_ms_sql import ReadCsv as rcsv
from calculation import Calculations as cal
from ploting import Plot as plt
##Read Csv files for train test and ideal data with in the same forlder
server = 'NASIR'  # e.g., 'localhost\SQLEXPRESS' or an IP address
database_name = 'Assignment'  # Connect to the database to create another database or just remove the database from the connection string
username = 'nasir'  # SQL Server username, omit if using Windows authentication
password = 'Cheema01'  # SQL Server password, omit if using Windows authentication
driver  = 'ODBC Driver 17 for SQL Server'
port = '1433'
dataset_path = r'C:\Users\nasir\OneDrive\Desktop\IU-Course Data\PROGRAMMING WITH PYTHON-course2-sem1\3.WRITTEN ASSIGNMENT\written Assignment\Assignment'
file_names = ['train.csv','test.csv','ideal.csv']
tabels_dic = {
            "train_table": [('X', 'FLOAT'), ('Y1 (training func)', 'FLOAT'), ('Y2 (training func)', 'FLOAT'), ('Y3 (training func)', 'FLOAT'), ('Y4 (training func)', 'FLOAT')],
            "test_table": [('X (test func)', 'FLOAT'), ('Y (test func)', 'FLOAT'), ('Delta Y (test func)', 'FLOAT'), ('No. of ideal func', 'VARCHAR(255)')],
            "ideal_table": [('X', 'FLOAT')] + [(f'Y{i} (ideal func)', 'FLOAT') for i in range(1, 51)],
        }
file_to_table_map = {
    'train.csv': 'train_table',
    'test.csv': 'test_table',
    'ideal.csv': 'ideal_table',
}
db_creator = cdt( server, 
                database_name, 
                username, 
                password, 
                driver, 
                port,
                tabels_dic)
db_copy =    rcsv( server, 
                database_name, 
                username, 
                password, 
                driver, 
                port,
                dataset_path,
                file_names,
                tabels_dic,
                file_to_table_map)


db_creator.create_database()
db_creator.create_tables()
#print(db_creator.create_tables())
db_copy.read_csv_to_sql()

# Initialize the engine
db_copy.alchemy_connection()

# Now, you can use db_conn.engine to perform database operations
engine = db_copy.engine

# Example: Reading data into a pandas DataFrame
df_train = pd.read_sql_query("SELECT * FROM train_table", engine)
df_ideal = pd.read_sql_query("SELECT * FROM ideal_table", engine)
df_test = pd.read_sql_query("SELECT * FROM test_table", engine)

# Example usage
#print(df_train.head())
#print(df_ideal.head())
#print(df_test.head())

# Create an instance of Class Calculations
calculations = cal(df_train, df_ideal, df_test)

# Calculate SSD sums and find top four ideal functions
calculations.calculate_criteria1()

# Access the ssd_sums and top_four_ideal_functions directly from the instance
ssd_sums = calculations.get_ssd_sums()
top_four_ideal_functions = calculations.get_top_four_ideal_functions()

#print("SSD sums for all comparisons:", ssd_sums)
#print("Top ideal function for each training function:", top_four_ideal_functions)

# Calculate Daviations
calculations.deviations()
adjusted_deviations = calculations.get_adjusted_deviation()
#Calculate Final Results base on df_test
calculations.results()

#Access Test Resutls
test_results = calculations.get_test_results()

# Create a new DataFrame from the results
df_test_results = pd.DataFrame(test_results)
df_test_results = df_test_results.sort_values(by='X (test func)')

#Write the test Data to test_results table in Microsoft SQL Server
# Initialize the engine
#db_copy.alchemy_connection()

# Now, you can use db_conn.engine to perform database operations
engine = db_copy.engine
table_name = 'test_results'
        
# Save the DataFrame to the SQL table
df_test_results.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
print(f'Data Copied to {table_name} in SQL')

# Create an instance of Class PlotSsdSums
ssd = plt(ssd_sums,df_test_results)

#Plot the DashBoard
ssd.dashboard()
#ssd.ssd_plot_only()
#ssd.scatter_plot_only()


In [ ]:
print(adjusted_deviations)